In [326]:
## Main SWR detection analysis! Note there are two ways to analyze the data: a cluster and local version. I keep them
## both updated with changes for now since I tend to check with the local version even if the cluster version is better
## for bulk analysis 2020-6-17 JS

In [145]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
HPC_labels = [MTL_labels[i] for i in [0,1,2,3,4,8,9,10,11,12,20,24,25]] # all labels within HPC
ENT_labels = [MTL_labels[i] for i in [6,14,22,26,28]] # all labels within entorhinal
PHC_labels = [MTL_labels[i] for i in [7,15,21]] # all labels within parahippocampal
exp = 'FR1'
poss_subs = df[df.experiment == exp]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
# sub lists I've been using for sample code

# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus
# subs = subs+['R1001P','R1092J','R1151E','R1320D'] # additional subs with issues that I resolved with various RR and corr thresholds
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2
# subs = ['R1112M','R1163T'] #'R1112M' # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1151E'] # all channels X sessions get removed but 3 after 60/180 hz line removal
# these should all be worked out now. See SWR FR1 problem sessions PPT on Box for details of each
# subs = ['R1299T','R1332M','R1397D','R1349T','R1339D','R1337E','R1275D','R1151E','R1123C','R1120E','R1045E'] # final surrounding_recall problem subs after looking at huge raster!
# subs = ['R1308T','R1338T','R1358T'] # these guys had weird SRs and some loading problems when I went to whole_retrieval analysis...fixed with rounding
# subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T'] # single session subs with memory allocation errors in ClusterRun
# subs = ['R1006P','R1010J','R1030J','R1032D','R1049J','R1051J','R1052E','R1054J','R1093J','R1098D','R1104D','R1108J','R1111M',
#         'R1115T','R1118N','R1124J','R1207J','R1230J','R1316T','R1329T','R1323T','R1337E','R1367D'] # subs that didn't load on 2020-07-04 encoding run
# subs = ['R1463E'] # test for localization.pairs
subs = ['R1286J','R1401J','R1406M']

bad_subs = ['R1433E','R1051J'] # no electrode cats

sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp)] # all sessions for subs
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[1:2]
sub_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
1545,NaN,protocols/r1/subjects/R1286J/experiments/FR1/s...,protocols/r1/subjects/R1286J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1286J/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1286J/localizations/0/m...,NaN,0,R1286J,R1286J,NaN,protocols/r1/subjects/R1286J/experiments/FR1/s...
1984,NaN,protocols/r1/subjects/R1401J/experiments/FR1/s...,protocols/r1/subjects/R1401J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1401J/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1401J/localizations/0/m...,NaN,0,R1401J,R1401J,3.3,protocols/r1/subjects/R1401J/experiments/FR1/s...
2006,NaN,protocols/r1/subjects/R1406M/experiments/FR1/s...,protocols/r1/subjects/R1406M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1406M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1406M/localizations/0/m...,NaN,3,R1406M,R1406M,3.3,protocols/r1/subjects/R1406M/experiments/FR1/s...
2007,NaN,protocols/r1/subjects/R1406M/experiments/FR1/s...,protocols/r1/subjects/R1406M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1406M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1406M/localizations/0/m...,NaN,4,R1406M,R1406M,3.3,protocols/r1/subjects/R1406M/experiments/FR1/s...


In [147]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
exp_df = exp_df[
                ((df.subject!='R1015J') | (df.session!=0)) & 
                ((df.subject!='R1063C') | (df.session!=1)) & 
                ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                ((df.subject!='R1120E') | (df.session!=0)) &
                ((df.subject!='R1122E') | (df.session!=2)) &
                ((df.subject!='R1154D') | (df.session!=0)) &
                ((df.subject!='R1186P') | (df.session!=0)) &
                ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                ((df.subject!='R1277J') | (df.session!=0)) &
                ((df.subject!='R1413D') | (df.session!=0)) & 
                ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                ((df.subject!='R1275D') | (df.session!=3)) # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
               ] 
exp_df.shape
exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(553, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
0,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,0,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
1,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...,protocols/r1/subjects/R1001P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1001P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1001P/localizations/0/m...,NaN,1,R1001P,R1001P,NaN,protocols/r1/subjects/R1001P/experiments/FR1/s...
9,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,0,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
10,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...,protocols/r1/subjects/R1002P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1002P/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1002P/localizations/0/m...,NaN,1,R1002P,R1002P,NaN,protocols/r1/subjects/R1002P/experiments/FR1/s...
17,NaN,protocols/r1/subjects/R1003P/experiments/FR1/s...,protocols/r1/subjects/R1003P/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1003P/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1003P/localizations/0/m...,NaN,0,R1003P,R1003P,1.0,protocols/r1/subjects/R1003P/experiments/FR1/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146,NaN,protocols/r1/subjects/R1462M/experiments/FR1/s...,protocols/r1/subjects/R1462M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1462M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1462M/localizations/0/m...,NaN,0,R1462M,R1462M,3.4,protocols/r1/subjects/R1462M/experiments/FR1/s...
2148,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,0,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,0,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2149,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,1,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,1,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2163,NaN,protocols/r1/subjects/R1466J/experiments/FR1/s...,protocols/r1/subjects/R1466J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1466J/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1466J/localizations/0/m...,NaN,0,R1466J,R1466J,3.4,protocols/r1/subjects/R1466J/experiments/FR1/s...


## LOCAL run (original program)

In [148]:
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload

### PARAMS ###

save_values = 1

selected_period = 'surrounding_recall' # surrounding_recall # whole_retrieval # encoding 
# there are three periods this code is set up to look at: periods aligned to recall, the entire retrieval period, and the encoding period
recall_type_switch = 0 # how do we select recalls?? Numbers 0:3
# 0: Original analysis taking only recalls without a recall in 2 s IRI before them
# 1: Take these same recalls, but keep only those WITH a recall within 2 s after they occur
# 2: test condition where we look at second recalls within IRI ONLY
# 3: ISOLATED only!

selected_region = HPC_labels #HPC_labels # ENT_labels+HPC_labels 

remove_soz_ictal = False # False only removes bad electrodes and not soz/ictal
min_ripple_rate = 0.1 # Hz.
max_ripple_rate = 1.0 # Hz.
max_trial_by_trial_correlation = 0.05 # if ripples correlated more than this remove them
max_electrode_by_electrode_correlation = 0.2 #???

filter_type = 'hamming' # 'butter' # Vaz used Butterworth from 80-120 Hz; Norman used Hamming from 70-180 Hz
# •Butterworth from Vaz et al: 2nd order from 80-120 ms, hilbert, select events >2 STD above mean of filtered traces.
#      Events >=25 ms long w/ max amp >3 SD were retained as ripples. Join adjacent ripples separated by <15 ms 
# •Hilbert from Norman et al: "70-180 Hz zero-lag linear-phase Hamming FIR filter w/ transition bandwidth of 5 Hz"
#      Then Hilbert, clip extreme to 4 SD, square this clipped, smooth w/ Kaiser FIR low-pass filter with 40 Hz cutoff,
#      mean and SD computed across entire experimental duration to define the threshold for event detection
#      Events from original (squared but unclipped) signal >4 SD above baseline were selected as candidate SWR events. 
#      Duration expanded until ripple power <2 SD. Events <20 ms or >200 ms excluded. Adjacent events <30 ms separation (peak-to-peak) merged.
# •Using IED detection from Vaz for 250 Hz highpass events (really 250-499). Norman uses 20-60 Hz events which is...odd.
#       See detectRipples code in module for this algorithm


# recall params
IRI = 2000 # inter-ripple interval...remove ripples within this range (keep only first one and remove those after it)
retrieval_whole_time = 30000
# encoding params
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.
eeg_buffer = 300 # buffer to add to either end of IRI when processing eeg #**

### END PARAMS ###

if 'entorhinal' in selected_region and 'left hippocampus' in selected_region:
    region_name = 'HPC_ENT'
elif 'entorhinal' in selected_region:
    region_name = 'ENT'
elif 'left hippocampus' in selected_region:
    region_name = 'HPC'
if selected_period == 'surrounding_recall':
    psth_start = -IRI # only makes sense to look at period <= IRI
    psth_end = IRI # how long to grab data after recall
elif selected_period == 'whole_retrieval':
    psth_start = -IRI # doesn't have to be IRI just 2000 ms is convenient
    psth_end = IRI+retrieval_whole_time
elif selected_period == 'encoding':
    psth_start = pre_encoding_time
    psth_end = encoding_time

ripple_array = []; fr_array = []; 
trial_nums = []; 
session_ct = 0; channel_ct = 0; total_channel_ct = 0
HPC_names = []; sub_names = []; sub_sess_names = []
electrodes_per_session = []
total_lists = 0; total_recalls = 0; kept_recalls = 0
ent_elec_ct = []; sd_regions = []; not_sd_regions = []
ripple_ied_accum_ct = []
time_add_save = [];             
encoded_word_key_array = []; serialpos_array = [] # encoding info
ordinal_position_array = []

trial_by_trial_correlation = []; elec_by_elec_correlation = []
elec_ripple_rate_array = []

all_regions = []
all_atlas = []
all_pair_numbers = []
all_stein_das = []

for row in exp_df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
        evs = reader.load('task_events')
        evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.recalled==True)] # recalled word AND correct (from this list...no instrusions). 
        
        # need to remove free recalls that happened more than once        
        word_evs = evs[evs['type']=='WORD'] # get words 
        evs_free_recall = removeRepeatedRecalls(evs_free_recall,word_evs)
        
        # select which recalls??         
        [recall_selection_name,selected_recalls_idxs] = selectRecallType(recall_type_switch,evs_free_recall,IRI)
        evs_free_recall = evs_free_recall[selected_recalls_idxs]
        
        # let's make sure remaining recalls have eeg 
        evs_free_recall = evs_free_recall[evs_free_recall.eegoffset > -1]
        if len(evs_free_recall)==0: #...and that any are left!
            break
            
        # get output position in each list for this session's recalls
        session_corrected_list_ops = getOutputPositions(evs,evs_free_recall)
            
        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
        except:
            localizations = []
        
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(tal_struct,localizations,pairs) 
        
        all_regions.extend(elec_regions)
        all_atlas.extend(atlas_type)
        all_pair_numbers.extend(pair_number)
        all_stein_das.append(has_stein_das)
        
        sub_sess_names.append(sub+'-'+str(session))
        
        
        
        
#         import ipdb; ipdb.set_trace()

#     #     # if you want to check regions labeled by stein/das v. those that weren't
#     #     sd_regions = np.append(sd_regions,elec_regions[atlas_type=='stein'])
#     #     sd_regions = np.append(sd_regions,elec_regions[atlas_type=='das'])
#     #     not_sd_regions = np.append(not_sd_regions,elec_regions[(atlas_type!='stein') & (atlas_type!='das')])

# #         # count elec regions with specific string...mostly here to comment out below and test for good sessions
# #         if len(findAinB(ENT_labels,elec_regions))>0:
# #             ent_ct = len(findAinB(ENT_labels,elec_regions))
# #             ent_elec_ct = np.append(ent_elec_ct,sub+'_'+str(session)+'_ct-'+str(ent_ct))

#         # load eeg
#         if selected_period == 'surrounding_recall':
#             total_recalls = total_recalls + len(evs_free_recall) # get total recalls from lists
#             total_lists = total_lists + len(evs[evs.type=='WORD'].list.unique()) # get total lists
#             kept_recalls = kept_recalls + len(evs_free_recall)
#             eeg_events = evs_free_recall
#         elif selected_period == 'whole_retrieval':
#             # grab whole retrieval periods for a better baseline of SWRs
#             evs_rets = evs[evs.type=='REC_START']
#             evs_rets = evs_rets[evs_rets.eegoffset>-1] # any trial with no eeg gets removed by cmlreaders so it's not in ripple_array 
#             evs_rets = evs_rets[evs_rets.list>-1] # remove practice lists
#             eeg_events = evs_rets
#         elif selected_period == 'encoding':
#             # I'm going to save encoding word events too, but need a mask to keep track of:
#             # 0) words not recalled 1) words recalled from this list 2) words later recalled BUT IRI<2 s so removed
#             evs_encoding_words = evs[evs.type=='WORD']
#             evs_encoding_words = evs_encoding_words[evs_encoding_words.eegoffset>-1]
#             encoded_word_key = np.zeros(len(evs_encoding_words)) # 0 for not recalled
#             encoded_word_key[evs_encoding_words.recalled==True] = 2 # 2 for recalled but removed bc IRI<2 s
#             encoded_word_key[evs_encoding_words.item_name.isin(evs_free_recall.item_name.unique())] = 1 # recalled words
#             # since finding all encoding words IN the list of correctly free recalled words won't have any intrusions
#             eeg_events = evs_encoding_words

#         if sub == 'R1045E': # this one session has issues in eeg trials past these points so remove events
#             if selected_period == 'surrounding_recall':
#                 eeg_events = eeg_events.iloc[:65,:] # only the first 66 recalls have good eeg
#             elif selected_period == 'whole_retrieval':
#                 eeg_events = eeg_events.iloc[:20,:] # only the first 20 retrieval periods have good eeg
#             elif selected_period == 'encoding':
#                 eeg_events = eeg_events.iloc[:263,:] # same idea
#                 encoded_word_key = encoded_word_key[:263]
            
#         eeg = reader.load_eeg(events=eeg_events, rel_start=psth_start-eeg_buffer, 
#                               rel_stop=psth_end+eeg_buffer, clean=True, scheme=pairs) #**
#         # events X electrodes X time
#         sr = eeg.samplerate

#         # if weird samplerate, add a few ms to make the load work
#         if (499<sr<500) | (998<sr<1000):
#             time_add = 1
#             if (499<sr<500):
#                 sr = 500
#             elif (998<sr<1000):
#                 sr = 1000
#             while eeg.shape[2] < (psth_end-psth_start+2*eeg_buffer)/(1000/sr):
#                 eeg = reader.load_eeg(events=eeg_events, rel_start=psth_start-eeg_buffer, 
#                                       rel_stop=psth_end+eeg_buffer+time_add, clean=True, scheme=pairs)
#                 if time_add>50: #**
#                     break
#                 time_add+=1
#             time_add_save.append(time_add)
#             eeg.samplerate = sr # need to overwrite those that were just fixed

#         eeg_ptsa = eeg.to_ptsa()
#         eeg = None # clear variable
# #         break;break;break # to look at eeg_ptsa (plots at bottom of notebook) before filtering

#         # line removal...don't do 120 for now
#         eeg_ptsa = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[58.,62.], filt_type='stop', order=4).filter()
#         eeg_ptsa = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[178.,182.], filt_type='stop', order=4).filter()
                
#         ## FILTERS ##
#         trans_width = 5.
#         ntaps = (2/3)*np.log10(1/(10*(1e-3*1e-4)))*(sr/trans_width) # gives 400 with sr=500, trans=5
#         if sr == 512 or sr == 1024:
#             ntaps = np.ceil(ntaps)
#         FIR_bandstop = firwin(int(ntaps+1), [70.,178.], fs=sr, window='hamming',pass_zero='bandstop')
#     #     FIR_bandpass = firwin(int(ntaps), [70.,180.], fs=sr, window='hamming',pass_zero='bandpass')
#         bandstop_25_60 = firwin(int(ntaps+1), [20.,58.], fs=sr, window='hamming',pass_zero='bandstop') # Norman 2019 IED
#     #     bandstop_60_80 = firwin(int(ntaps+1), [60.,80.], fs=sr, window='hamming',pass_zero='bandstop') # Gelinas 2016 IED
#         nyquist = sr/2
#         ntaps40, beta40 = kaiserord(40, trans_width/nyquist)
#         kaiser_40lp_filter = firwin(ntaps40, cutoff=40, window=('kaiser', beta40), scale=False, nyq=nyquist, pass_zero='lowpass')

#         # filter for ripples using filter selected above

#         if filter_type == 'hamming':
#             # need to subtract out to get the filtered signal since default is bandstop but want to keep it as PTSA        
#             eeg_rip_band = eeg_ptsa-filtfilt(FIR_bandstop,1.,eeg_ptsa) 
#             #         eeg_rip_band = filtfilt(FIR_bandpass,1.,eeg_ptsa) # can't use ptsa_to_mne this way so use eeg-bandstopped signal
#             eeg_ied_band = eeg_ptsa-filtfilt(bandstop_25_60,1.,eeg_ptsa)

#         elif filter_type == 'butter':
#             eeg_rip_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[80.,120.], filt_type='bandpass',order=2).filter()
#             if sr == 500: # dropped below 250 Hz because too close Nyquist
#                 eeg_ied_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=200., filt_type='highpass',order=2).filter() 
#             elif sr >= 1000:    
#                 #this seems okay since large range far from Nyquist...problem is it likely misses key events
#                 eeg_ied_band = ButterworthFilter(timeseries=eeg_ptsa, freq_range=[250.,499.], filt_type='bandpass',order=2).filter()
#             eeg_raw = ptsa_to_mne(eeg_ptsa,[0,psth_end-psth_start+2*eeg_buffer])    #**  
#         eeg_ptsa = None # clear variable

#         eeg_rip_band = ptsa_to_mne(eeg_rip_band,[0,psth_end-psth_start+2*eeg_buffer]) #** 
#         _ = eeg_rip_band.apply_hilbert(envelope=True)
#         eeg_ied_band = ptsa_to_mne(eeg_ied_band,[0,psth_end-psth_start+2*eeg_buffer]) #** 
#         _ = eeg_ied_band.apply_hilbert(envelope=True)
        
#         # get bad channel mask
#         try:
#             elec_cats = reader.load('electrode_categories') # this is cool
#         except:
#             if remove_soz_ictal == True:
#                 e = 'No electrode categories for '+sub+', session '+str(session)
#                 LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt')
#                 continue
#             else:
#                 elec_cats = []
            
#         bad_bp_mask = getBadChannels(tal_struct,elec_cats,remove_soz_ictal)
        
#         electrode_search_range = range(len(elec_regions))
#         if sub == 'R1120E':
#             electrode_search_range = range(30) # HPC elecs after 25:26 have lots of artifacts that get picked up as SWRs. See subject figure PPT
#         elif sub == 'R1349T': # channels 90 and below have tons of artifcats. After that looks okay though
#             electrode_search_range = range(91,len(elec_regions))
#         elif sub == 'R1397D': # for these two sessions two pairs of the electrodes have lots of correlated noise. Remove them
#             electrode_search_range = [i for i in range(len(elec_regions)) if i != 60]
#             electrode_search_range.remove(110)
#         elif sub == 'R1332M' and session == 1:
#             electrode_search_range = [i for i in range(len(elec_regions)) if i != 48] # some real weird bands in these couple sessions
#             electrode_search_range.remove(49)
#         elif sub == 'R1299T':
#             electrode_search_range = [i for i in range(len(elec_regions)) if i != 34] # see PPT. These two pairs shared an electrode and had tons of correlated artifacts
#             electrode_search_range.remove(43)

#         region_electrode_ct = 0
#         session_ripple_rate_by_elec = [];
#         for channel in electrode_search_range:
#             if (elec_regions[channel] in selected_region) & \
#             (bad_bp_mask[channel] == 0): # don't use bad electrodes
                
#                 total_channel_ct+=1 # total channels before artifact removal

#                 # get data from MNE container 
#                 eeg_rip = eeg_rip_band.get_data()[:,channel,:]     
#                 eeg_ied = eeg_ied_band.get_data()[:,channel,:]

#                 # select detection algorithm (note that iedlogic is same for both so always run that)
                
#                 if filter_type == 'hamming':
#                     # filter IEDs 
#                     eeg_ied = eeg_ied**2 # already rectified now square
#                     eeg_ied = filtfilt(kaiser_40lp_filter,1.,eeg_ied) # low pass filter  
#                     mean1 = np.mean(eeg_ied)
#                     std1 = np.std(eeg_ied)
#                     iedlogic = eeg_ied>=mean1+4*std1 # Norman et al 2016                
#                     # detect ripples
#                     ripplelogic = detectRipplesHamming(eeg_rip,trans_width,sr,iedlogic)
#                 elif filter_type == 'butter':
#                     eeg_mne = eeg_raw.get_data()[:,channel,:]
#                     # detect ripples
#                     ripplelogic, iedlogic = detectRipplesButter(eeg_rip,eeg_ied,eeg_mne,sr)             

# #                 ripple_ied_accum_ct.append([sub,session,sr,sum(ripplelogic),sum(iedlogic),sum(ripplelogic2),sum(iedlogic2),
# #                                             sum(ripplelogic_enc),sum(iedlogic_enc)])

#                 if sr>desired_sample_rate: # downsampling here for anything greater than 500
#                     ripplelogic = downsampleBinary(ripplelogic,sr/desired_sample_rate)
            
#                 # ripples are detected, so can remove buffers now #**
#                 ripplelogic = ripplelogic[:,int(eeg_buffer/(1000/desired_sample_rate)):
#                                           int((psth_end-psth_start+eeg_buffer)/(1000/desired_sample_rate))]
    
#                 # skip this electrode if the ripple rate is below threshold
#                 temp_start_array,_ = getStartEndArrays(ripplelogic)
#                 elec_ripple_rate = sum(temp_start_array)/temp_start_array.shape[0]/((psth_end-psth_start)/1000)
#                 if elec_ripple_rate < min_ripple_rate:
#                     print(sub+', '+str(session)+' skipped b/c below ripple rate thresh for ch.: '+str(channel))
#                     continue
#                 elif elec_ripple_rate > max_ripple_rate:
#                     print(sub+', '+str(session)+' skipped b/c ABOVE ripple rate thresh for ch.: '+str(channel))
#                     continue # skip this electrode
    
#                 # check the ripples for this electrode and make sure they're not super correlated across trials

#                 # first, bin the array so can get more realistic correlation not dependent on ms timing
#                 binned_ripplelogic = downsampleBinary(ripplelogic,10) # downsample by 10x so 10 ms bins
#                 trial_ripple_df = pd.DataFrame(data=np.transpose(binned_ripplelogic))
#                 num_cols = len(list(trial_ripple_df))
#                 trial_ripple_df.columns = ['col_' + str(i) for i in range(num_cols)] # generate range of ints for suffixes
#                 temp_tbt_corr = np.mean(pg.pairwise_corr(trial_ripple_df,method='spearman').r)
#                 if temp_tbt_corr > max_trial_by_trial_correlation:
#                     print(sub+', '+str(session)+' skipped b/c above trial-by-trial correlation for ch.: '+str(channel))
#                     continue

#                 ## if this electrode passes SAVE data ##
                
#                 trial_by_trial_correlation.append(temp_tbt_corr) # corr b/w trials
#                 elec_ripple_rate_array.append(elec_ripple_rate) # record the average ripple rate for this electrode
                
#                 # append arrays across electrodes
#                 ripple_array = superVstack(ripple_array,ripplelogic) # append across electrodes
#                 session_ripple_rate_by_elec = superVstack(session_ripple_rate_by_elec,np.mean(binned_ripplelogic,0)) # for correlation b/w elecs below
                
#                 if selected_period == 'encoding':
#                     encoded_word_key_array.append(encoded_word_key) # save the key for each electrode so easier to unpack later
#                     serialpos_array.append(evs_encoding_words.serialpos)
#                     if len(encoded_word_key) != ripplelogic.shape[0]:
#                         e = 'Encoded word key and ripple_array dont match for '+sub+str(session)+'_'+str(channel)
#                         LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt')
                        
#                 if selected_period == 'whole_retrieval':
#                     # make a matrix of the times of free recall if looking at whole retrieval period
#                     list_nums = evs_rets.list.unique() # count all list numbers for this session
#                     temp_fr_array = np.zeros((len(list_nums),retrieval_whole_time))
#                     for ln in list_nums:
#                         list_times = evs_free_recall[evs_free_recall.list==ln].rectime # I confirmed these are accurate relative to REC_START
#                         temp_fr_array[ln-list_nums[0]][np.array(list_times)] = 1 # subtract from list_nums[0] to recenter fr_array on 0 trial           
#                     fr_array = superVstack(fr_array,temp_fr_array)
                
#                 channel_ct+=1 # total channels from beginning 
#                 region_electrode_ct+=1 # channel ct for this session
#                 HPC_names.append(elec_regions[channel])
#                 sub_names.append(sub)
#                 sub_sess_names.append(sub+'-'+str(session))
#                 print('Cumulative array size after channel: '+str(channel))
#                 print(ripple_array.shape)
#                 trial_nums = np.append(trial_nums,ripplelogic.shape[0])
                
#                 # additional check to make sure eeg was recorded for all events
#                 if selected_period == 'surrounding_recall':
#                     if evs_free_recall.shape[0] != ripplelogic.shape[0]:
#                         # this happens because eeg wasn't turned on. remove eegoffset=-1 trials; I DO THIS ABOVE NOW
#                         good_evs_free_recall = evs_free_recall.eegoffset > -1
#                         session_corrected_list_ops = session_corrected_list_ops[good_evs_free_recall]
#                         e = sub+str(session)+'_still unrecorded eegs'
#                         LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt')

#                 ordinal_position_array.extend(session_corrected_list_ops) # update for each electrode
                      
#         # before we keep all the electrodes in this session, remove electrodes that are highly correlated.
#         # this really only makes sense for the cluster version, since I only do a session at a time here
#         if np.sum(session_ripple_rate_by_elec)>0 and region_electrode_ct>1:
#             session_ripple_df = pd.DataFrame(data=np.transpose(session_ripple_rate_by_elec))
#             num_cols = len(list(session_ripple_df))
#             session_ripple_df.columns = ['col_' + str(i) for i in range(num_cols)] # generate range of ints for suffixes
# #             print(pg.pairwise_corr(session_ripple_df,method='spearman').r) # print the elec-by-elec correlations
#             elec_by_elec_correlation = np.mean(pg.pairwise_corr(session_ripple_df,method='spearman').r) # correlation b/w elecs
#             if elec_by_elec_correlation > max_electrode_by_electrode_correlation:
#                 print('Correlation b/w electrodes too high, corr = '+str(elec_by_elec_correlation))
#             else:
#                 print('good set of electrodes (not correlated with each other)')

#         electrodes_per_session = np.append(electrodes_per_session,region_electrode_ct)
#         session_ct+=1
        
    except Exception as e:
        LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
        continue

# csvWriter(ripple_ied_accum_ct,'ieds') # if you want to save the accumulated ripple numbers

# if save_values == 1:
#     if remove_soz_ictal == False:
#         soz_label = 'soz_in'
#     else:
#         soz_label = 'soz_out'
#     if session_ct == 1:
#         fn = os.path.join('/home1/john/SWR/compiled/',
#             'SWR_'+region_name+'_'+str(session_ct)+'_sessions_'+str(len(np.unique(sub_names)))+'subjects_'+selected_period+recall_selection_name+'_'+soz_label+'.p')            
#     else:
#         fn = os.path.join('/home1/john/SWR/compiled/',
#             'SWR_'+region_name+'_'+str(session_ct)+'_sessions_'+str(len(np.unique(sub_names)))+'subjects_'+selected_period+recall_selection_name+'_'+soz_label+'.p')            
#     with open(fn,'wb') as f:
#         pickle.dump({'channel_ct':channel_ct, 'session_ct':session_ct,'total_channel_ct':total_channel_ct,
#                     'HPC_names':HPC_names, 'sub_sess_names':sub_sess_names,
#                     'ripple_array':ripple_array, #'ripple_array2':ripple_array2, 'ripple_array_enc':ripple_array_enc,
#                     'trial_nums':trial_nums, #'trial_nums2':trial_nums2,'trial_nums_enc':trial_nums_enc,
#                     'fr_array':fr_array, 'sub_names':sub_names, 
#                     'encoded_word_key_array':encoded_word_key_array,'serialpos_array':serialpos_array,
#                     'total_recalls':total_recalls, 'kept_recalls':kept_recalls,
#                     'trial_by_trial_correlation':trial_by_trial_correlation,'elec_ripple_rate_array':elec_ripple_rate_array}, f)


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1001P/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1001P/tal/R1001P_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data

2020-08-17_17-04-31: DF Exception: Sub: R1093J, Sess: 0, TypeError, cannot convert the series to <class 'int'>, file: <ipython-input-148-4a3626a6d68f>, line no: 96


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1094T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1094T/tal/R1094T_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/readers/electrodes.py:236: MissingCoordinatesWarning: Could not load MNI coordinates
  "Could not load MNI coordinates"))
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1094T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1094T/tal/R1094T_ta

2020-08-17_17-05-26: DF Exception: Sub: R1171M, Sess: 2, TypeError, cannot convert the series to <class 'int'>, file: <ipython-input-148-4a3626a6d68f>, line no: 96


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1172E/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1172E/tal/R1172E_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1172E/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1172E/tal/R1172E_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data

...

/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1329T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1329T/tal/R1329T_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1330D/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1330D/tal/R1330D_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data

2020-08-17_17-06-54: DF Exception: Sub: R1331T, Sess: 0, TypeError, cannot convert the series to <class 'int'>, file: <ipython-input-148-4a3626a6d68f>, line no: 96


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1331T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1331T/tal/R1331T_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1332M/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1332M/tal/R1332M_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data

2020-08-17_17-07-05: DF Exception: Sub: R1341T, Sess: 1, TypeError, cannot convert the series to <class 'int'>, file: <ipython-input-148-4a3626a6d68f>, line no: 96


/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1341T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1341T/tal/R1341T_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1341T/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1341T/tal/R1341T_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data

In [151]:
len(all_atlas)
len(all_regions)
np.unique(all_atlas)
np.unique(all_regions)

62046

62046

array(['MTL_localization', 'das', 'dk', 'dk_localization', 'ind', 'stein',
       'wb', 'wb_localization'], dtype='<U16')

array([' left acgg anterior cingulate gyrus',
       ' left ains anterior insula', ' left amygdala',
       ' left ang angular gyrus', ' left aorg anterior orbital gyrus',
       ' left calc calcarine cortex', ' left caudate',
       ' left cerebellum exterior', ' left cerebral white matter',
       ' left co central operculum', ' left cun cuneus',
       ' left ent entorhinal area', ' left fo frontal operculum',
       ' left frp frontal pole', ' left fug fusiform gyrus',
       ' left gre gyrus rectus', ' left hippocampus',
       ' left inf lat vent', ' left iog inferior occipital gyrus',
       ' left itg inferior temporal gyrus', ' left lateral ventricle',
       ' left lig lingual gyrus', ' left lorg lateral orbital gyrus',
       ' left mcgg middle cingulate gyrus',
       ' left mfc medial frontal cortex',
       ' left mfg middle frontal gyrus',
       ' left mog middle occipital gyrus',
       ' left morg medial orbital gyrus',
       ' left mpog postcentral gyrus medial segm

In [160]:
all_regions[0]

'rostralmiddlefrontal'